# <font color="purple">Adversarial Training of Residual Network</font>

Definition of WideResNet and Architecture:

##  <center>The Result of Residual Network </center>

***Road Map***
* Data Preprocessing
* Model Cross Validation Results
* Evaluate the GridSearchCV Results
* Model Training and Learning Curves
* Model Adversarial Training Approach.


**Libraries**

In [1]:
!pip install -qq -e git+http://github.com/tensorflow/cleverhans.git#egg=cleverhans
import sys
sys.path.append('/content/src/cleverhans')
sys.path.append('cleverhans')
sys.path.insert(1, '/src/cleverhans')

In [2]:

sys.path.insert(2, '/home/sefika/adversarial_examples_parseval_net/src/models')
sys.path.insert(3, '/home/sefika/adversarial_examples_parseval_net/src/models/widesesNet')
sys.path.insert(4, '/home/sefika/adversarial_examples_parseval_net/src/models/Parseval_Networks')
sys.path.insert(5, '/home/sefika/adversarial_examples_parseval_net/src/train')
sys.path.insert(6, '/home/sefika/adversarial_examples_parseval_net/src/preprocessing')


In [3]:
import tensorflow as tf
from cleverhans.future.tf2.attacks import fast_gradient_method
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import  KFold
import gzip
import pickle
import numpy as np
from parsevalnet import ParsevalNetwork
import warnings
warnings.filterwarnings("ignore")

print("\nTensorflow Version: " + tf.__version__)
# utility functions
from preprocessing import preprocessing_data
# Define configuration parameters
from _utility import lrate
from training import train
from adversarial_training import  AdversarialTraining


Tensorflow Version: 2.4.0-dev20201021


## <font color="green"> Data Preprocessing </font>


* Read Data from File

In [4]:
def read_data():
    with open("/home/sefika/adversarial_examples_parseval_net/data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    return data
data = read_data()

* Call data preprocessing function

In [5]:
X, y = preprocessing_data(data)
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size = 0.1)
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.1)

## <font color="green"> Utilize Functions </font>

* Flipping the image using data augmentation technique

In [6]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)

### <font color = "green">Some Parameters Regarding Adversarial Examples</font>

In [7]:
# predefined epsilon values
epsilon_list = [i/1000 for i in range(1,33)]

In [8]:
EPOCHS = 50
BS = 64
init = (32, 32,1)
sgd = SGD(lr=0.1, momentum=0.9)
parameter = {'epochs': EPOCHS, 'batch_size': BS, 'optimizer': sgd}

In [9]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
callbacks_list = [lrate, es]
parseval = ParsevalNetwork(init,0.0001, 0.9, nb_classes=4, N=2, k=1, dropout=0.0)

In [10]:
premodel = parseval.create_wide_residual_network()
premodel.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["acc"])
premodel.fit(generator.flow(x_train, y_train, batch_size=BS),steps_per_epoch=len(x_train) // BS, 
              epochs= EPOCHS, callbacks = callbacks_list,
              validation_data=(x_val, y_val),
              validation_steps=x_val.shape[0] // BS,
              )

Parseval  Network-16-1 created.


KeyboardInterrupt: 

In [ ]:
import numpy as np
from cleverhans.future.tf2.attacks import fast_gradient_method
import pandas as pd
from sklearn.model_selection import KFold
import sys
import tensorflow
import tensorflow as tf

from _utility import print_test, get_adversarial_examples

import pickle
folder_name = "/home/sefika/adversarial_examples_parseval_net/src/logs/saved_models/"

def train(instance, X_train, Y_train, X_test, y_test, epochs,
BS, sgd, generator, callbacks_list, model_name="ResNet"):

  
    kfold = KFold(n_splits=10, random_state=42, shuffle=False)
    for j, (train, val) in enumerate(kfold.split(X_train)):
        model = instance.create_wide_residual_network()
        model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["acc"])
        print("Finished compiling")
        x_train, y_train = X_train[train], Y_train[train]
        x_val, y_val = X_train[val], Y_train[val]

        hist = model.fit(
            generator.flow(x_train, y_train, batch_size=BS),
            steps_per_epoch=len(x_train) // BS,
            epochs= epochs, callbacks = callbacks_list,
            validation_data=(x_val, y_val),
            validation_steps=x_val.shape[0] // BS,
            )
                ## write the history
        with open('history_'+model_name+str(j), 'wb') as file_pi:
            pickle.dump(hist.history, file_pi)
    
        name = folder_name+model_name+"_"+str(j)+".h5"
        model.save_weights(name)


In [ ]:
train(parseval, X_train, Y_train, X_test, y_test, EPOCHS, BS, sgd, generator, callbacks_list, model_name="Parseval")

In [ ]:
folder = "/home/sefika/adversarial_examples_parseval_net/src/logs/saved_models/"
import os
from os import listdir
from os.path import isfile, join

files = os.listdir(folder)
files

In [ ]:
for i, file in enumerate(files):
    model = tf.keras.models.load_model(folder+file)
    model.evaluate(X_test, y_test)

## <font color="purple">2.) Adversarial Training on the Model</font>

In [31]:
import numpy as np
from cleverhans.future.tf2.attacks import fast_gradient_method
import pandas as pd
from sklearn.model_selection import KFold
import sys
import tensorflow
import tensorflow as tf
#sys.path.insert(1, '/home/sefika/AE_Parseval_Network/src/models')
import gc

from _utility import print_test, get_adversarial_examples

import pickle

class AdversarialTraining(object):
    """[summary]

    Args:
        object ([type]): [description]
    """
    def __init__(self, parameter):
        self.epochs = parameter['epochs']
        self.batch_size = parameter['batch_size']
        self.optimizer = parameter['optimizer']

        self.generator = tensorflow.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=10,
            width_shift_range=5. / 32,
            height_shift_range=5. / 32,
        )

    def train(self,
              instance,
              X_train,
              Y_train,
              X_test,
              y_test,
              epsilon_list,
              callbacks_list,
              model_name="ResNet"):
        """[summary]

        Args:
            instance ([type]): [description]
            X_train ([type]): [description]
            Y_train ([type]): [description]
            X_test ([type]): [description]
            y_test ([type]): [description]
            epsilon_list ([type]): [description]
            callbacks_list ([type]): [description]
            model_name (str, optional): [description]. Defaults to "ResNet".

        Returns:
            [type]: [description]
        """

        # Ten fold cross validation

        kfold = KFold(n_splits=10, random_state=42, shuffle=False)

        for j, (train, val) in enumerate(kfold.split(X_train)):

            model = instance.create_wide_residual_network()
            model.compile(loss="categorical_crossentropy",
                          optimizer=self.optimizer,
                          metrics=["acc"])
            print("Finished compiling")
            print(epsilon_list)
            x_train, y_train = X_train[train],Y_train[train]
            x_val, y_val = X_train[val], Y_train[val]
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            train_dataset = train_dataset.batch(self.batch_size)
            val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            val_dataset = val_dataset.batch(self.batch_size)

            for epoch in range(self.epochs):
                for step, (x_train, y_train) in enumerate(train_dataset):
                    x_train = self.data_augmentation(x_train, y_train, model, epsilon_list)
                    model.fit(x_train, y_train)
                    print(step)
            name = model_name+"_"+str(j)+".h5"
            model.save_weights(name)
            
           

    def data_augmentation(self, X_train, Y_train, pretrained_model,
                          epsilon_list):
        """[summary]

        Args:
            X_train ([type]): [description]
            Y_train ([type]): [description]
            pretrained_model ([type]): [description]
            epsilon_list ([type]): [description]

        Returns:
            [type]: [description]
        """
        first_half_end = int(len(X_train) / 2)
        second_half_end = int(len(X_train))
        x_clean = X_train[0:first_half_end, :, :, :]
        x_adv = self.get_adversarial(
            pretrained_model, X_train[first_half_end:second_half_end, :, :, :],
            Y_train[first_half_end:second_half_end], epsilon_list)
        x_mix = self.merge_data(x_clean, x_adv)
        y_mix = Y_train[0:second_half_end]

        return x_mix, y_mix

    def merge_data(self, x_clean, x_adv):
        """[summary]

        Args:
            x_clean ([type]): [description]
            x_adv ([type]): [description]

        Returns:
            [type]: [description]
        """
        x_mix = []
        for i in range(len(x_clean)):
            x_mix.append(x_clean[i])
        for j in range(len(x_adv)):
            x_mix.append(x_adv[j])
        x_mix = np.array(x_mix)

        return x_mix

    def get_adversarial(self, logits_model, X_true, y_true, epsilon_list):
        return self.adversarial_example(logits_model, X_true, y_true,
                                        epsilon_list)

    def adversarial_example(self, logits_model, X_true, y_true, epsilon_list):
        X_adv = []

        for index, x_true  in enumerate(X_true):
            epsilon = epsilon_list[index]
          
            original_image = x_true
            original_image = tf.reshape(original_image, (1, 32, 32))
            original_label = y_true[index]
            original_label = np.reshape(np.argmax(original_label),
                                      (1, )).astype('int64')
            adv_example_targeted_label = fast_gradient_method(
                logits_model,
                original_image,
                epsilon,
                np.inf,
                y=original_label,
                targeted=False)
            X_adv.append(
                np.array(adv_example_targeted_label).reshape(32, 32, 1))
        X_adv = np.array(X_adv)
        return X_adv

In [32]:
adversarial_training =  AdversarialTraining(parameter)
adversarial_training.train(parseval, X_train, Y_train, X_test, y_test, epsilon_list, callbacks_list,model_name="parseval")

Parseval  Network-16-1 created.
Finished compiling
[0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019, 0.02, 0.021, 0.022, 0.023, 0.024, 0.025, 0.026, 0.027, 0.028, 0.029, 0.03, 0.031, 0.032]


ValueError: in user code:

    /home/sefika/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/sefika/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/sefika/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/sefika/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/sefika/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3401 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/sefika/anaconda3/lib/python3.7/site-packages/tensorflow/py

In [ ]:
result_adv_df

### <font color="purple"> Results of Adversarial Training</font>

# <font color="blue">Compare Non-Adversarial Training with Adversarial Training</font>

In [ ]:
result_adv_df[column].head(1)

In [ ]:
result_df[column].head(1)

# <font color="blue">Conclusion</font>